### Example 3.10: 전이학습

- Example 3.9의 모델을 이용해서 추가적인 학습을 시키는 경우

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam

# MNIST 데이터셋을 읽고 신경망에 입력할 형태로 변환
(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

In [ ]:
cnn_model = tf.keras.models.load_model('saved_cnn.h5')

OSError: ignored

In [ ]:
res_cnn=cnn_model.evaluate(x_test,y_test,verbose=0)
print("1. cnn model 정확률은",res_cnn[1]*100)
print('   res_cnn = ', res_cnn,)

NameError: ignored

In [ ]:
for curr_layer in cnn_model.layers:
    print(curr_layer.trainable, ':==', curr_layer)
    curr_layer.trainable = False

no_layer = len(cnn_model.layers)
cnn_model.layers[no_layer-3].trainable = True
cnn_model.layers[no_layer-2].trainable = True
cnn_model.layers[no_layer-1].trainable = True

print("\nLayers.trainable = ", end = ' ')
for curr_layer in cnn_model.layers:
    print(curr_layer.trainable, end = ' ')

True :== <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000204C86EE640>
True :== <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000204C431EB20>
True :== <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000204C42F4370>
True :== <tensorflow.python.keras.layers.core.Dropout object at 0x00000204C3B50640>
True :== <tensorflow.python.keras.layers.core.Flatten object at 0x00000204C3B4E4C0>
True :== <tensorflow.python.keras.layers.core.Dense object at 0x00000204C3B47DF0>
True :== <tensorflow.python.keras.layers.core.Dropout object at 0x00000204C3B477F0>
True :== <tensorflow.python.keras.layers.core.Dense object at 0x00000204C3B3F430>

Layers.trainable =  False False False False False True True True 

In [ ]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [ ]:
cnn_model.compile(loss="categorical_crossentropy",
              optimizer=Adam(0.002),
              metrics=["accuracy"])

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    'transfer_best_cnn.h5', monitor = 'val_accuracy', save_best_only=True)

hist=cnn_model.fit(x_train,y_train,batch_size=128,epochs=10,
             validation_data=(x_test,y_test),
             callbacks = [checkpoint_cb], verbose=2)

Epoch 1/10
469/469 - 3s - loss: 0.1884 - accuracy: 0.9298 - val_loss: 0.2244 - val_accuracy: 0.9226
Epoch 2/10
469/469 - 2s - loss: 0.1736 - accuracy: 0.9354 - val_loss: 0.2251 - val_accuracy: 0.9238
Epoch 3/10
469/469 - 2s - loss: 0.1647 - accuracy: 0.9392 - val_loss: 0.2298 - val_accuracy: 0.9225
Epoch 4/10
469/469 - 2s - loss: 0.1529 - accuracy: 0.9422 - val_loss: 0.2424 - val_accuracy: 0.9244
Epoch 5/10
469/469 - 2s - loss: 0.1457 - accuracy: 0.9450 - val_loss: 0.2387 - val_accuracy: 0.9261
Epoch 6/10
469/469 - 2s - loss: 0.1391 - accuracy: 0.9469 - val_loss: 0.2418 - val_accuracy: 0.9246
Epoch 7/10
469/469 - 2s - loss: 0.1315 - accuracy: 0.9488 - val_loss: 0.2447 - val_accuracy: 0.9258
Epoch 8/10
469/469 - 2s - loss: 0.1288 - accuracy: 0.9504 - val_loss: 0.2445 - val_accuracy: 0.9245
Epoch 9/10
469/469 - 2s - loss: 0.1230 - accuracy: 0.9531 - val_loss: 0.2367 - val_accuracy: 0.9298
Epoch 10/10
469/469 - 2s - loss: 0.1190 - accuracy: 0.9545 - val_loss: 0.2493 - val_accuracy: 0.9246

In [ ]:
res_transfer_cnn=cnn_model.evaluate(x_test,y_test,verbose=0)
print("2. transfered learning model 정확률은",res_transfer_cnn[1]*100)
print('   res_cnn = ', res_transfer_cnn,'\n')

2. transfered learning model 정확률은 92.46000051498413
   res_cnn =  [0.24929852783679962, 0.9246000051498413] 



### Example 3-11: Model의 구조를 변경하기

- 원래는 10개의 fashion item이 있음.
- 필요에 의해 5개의 fashion item만 인식하는 경우
   - 기존에 학습된 것을 이용하고, 5개의 fashion item에 대해 학습하기

In [ ]:
cnn_model = tf.keras.models.load_model('saved_cnn.h5')
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [ ]:
cnn_model.pop()
cnn_model.pop()
cnn_model.pop()
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cnn_model.add(Dense(64,activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(5,activation='softmax'))
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                589888    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0